# Prerequisites

## Accessing ADLS using Access keys

In [0]:
storage_account = "azurestgacctraining"
container = "users"
users_file_path = "/raw/*.csv"
storage_account_conf_key = f"fs.azure.account.key.{storage_account}.dfs.core.windows.net"
storage_account_conf_key_value = "IYpZg1qsiBaGyKWhbRUByYBNWAxovDP+rac+jqoDik5hFFqo6nwvzD+6xiFntCES4jTUGoOZMRil+ASt4lYSMQ=="

spark.conf.set(storage_account_conf_key,storage_account_conf_key_value)
users_ext_location = f"abfss://{container}@{storage_account}.dfs.core.windows.net/"
spark.read.csv(path=f"{users_external_location}{users_file_path}",
               header=True, inferSchema=True).limit(4).display()

id,name,dob,email,gender,country,region,city,asset,marital_status
1,Heather Gibbs,2024-10-31,heathergibbs6243@gmail.com,Female,United States,Virginia,Virginia Beach,734388,Married
2,Herrod Petersen,2024-02-19,herrodpetersen@yahoomail.com,Male,United States,Arizona,Phoenix,113506,Single
3,Ocean Workman,2024-10-10,oceanworkman2328@ymail.com,Male,United States,Tennessee,Clarksville,139985,Married
4,Xaviera Maxwell,2025-03-09,xavieramaxwell@gmail.com,Transgender,United States,Ohio,Cleveland,511409,Married


## External Location Path

In [0]:
users_external_location = spark.sql("DESCRIBE EXTERNAL LOCATION `users_ext_location`").select("url").collect()[0][0]
spark.read.csv(path=f"{users_external_location}/raw/*.csv",
               header=True,
               inferSchema=True).limit(4).display()

id,name,dob,email,gender,country,region,city,asset,marital_status
1,Heather Gibbs,2024-10-31,heathergibbs6243@gmail.com,Female,United States,Virginia,Virginia Beach,734388,Married
2,Herrod Petersen,2024-02-19,herrodpetersen@yahoomail.com,Male,United States,Arizona,Phoenix,113506,Single
3,Ocean Workman,2024-10-10,oceanworkman2328@ymail.com,Male,United States,Tennessee,Clarksville,139985,Married
4,Xaviera Maxwell,2025-03-09,xavieramaxwell@gmail.com,Transgender,United States,Ohio,Cleveland,511409,Married


## Create Catalog, Schema and Table

In [0]:
%sql
-- Execute only for Azure Databricks
DROP CATALOG IF EXISTS quickstart_catalog CASCADE;
CREATE CATALOG IF NOT EXISTS quickstart_catalog;
USE CATALOG quickstart_catalog;

-- Azure Databricks + Databricks Community Edition
CREATE SCHEMA IF NOT EXISTS quickstart_schema;
CREATE TABLE quickstart_schema.users (
  id INT,
  name STRING,
  dob DATE,
  email STRING,
  gender STRING,
  country STRING,
  region STRING,
  city STRING,
  asset INT,
  marital_status STRING
)

In [0]:
ingest()

---------------------------------------------------------------------------
PySparkValueError                         Traceback (most recent call last)
File <command-899214470609803>, line 1
----> 1 ingest()

File <command-899214470609801>, line 17, in ingest()
      1 def ingest():
      2   source_df = (spark.readStream
      3                       .format("cloudFiles")
      4                       .option("cloudFiles.format", "csv")  
   (...)
      9                                             .load(f"{users_external_location}{users_file_path}")
     10   )
     12   write_query = (source_df.writeStream
     13                           .format("delta")
     14                           .option("checkpointLocation", f"{users_ext_location}/chekpoint/")
     15                           .option("mergeSchema", "true")
     16                           .outputMode("append")                          
---> 17                           .trigger()
     18                           .toTab

In [0]:
from pyspark.sql.functions import col
spark.read.table("quickstart_schema.users").filter(col("id")>=5501).limit(4).display()

id,name,dob,email,gender,country,region,city,asset,marital_status,_rescued_data,education
5501,Chelsea Mcdowell,null,chelseamcdowell@ymail.com,Transgender,United States,Arkansas,Fort Smith,140903,Single,"{""dob"":""###"",""_file_path"":""abfss://users@azurestgacctraining.dfs.core.windows.net/raw/users_012_datatype_mismatch_dob.csv""}",Master's Degree


# Data Ingestion using Autoloader

In [0]:
def ingest():
  source_df = (spark.readStream
                      .format("cloudFiles")
                      .option("cloudFiles.format", "csv")  
                      .option("header", "true") 
                      .option("timestampFormat","yyyy-MM-dd")                  
                      .option("cloudFiles.schemaLocation", f"{users_ext_location}/chekpoint/schema")
                      .option("cloudFiles.inferColumnTypes", "true")
                                            .load(f"{users_external_location}{users_file_path}")
  )
  write_query = (source_df.writeStream
                          .format("delta")
                          .option("checkpointLocation", f"{users_ext_location}/chekpoint/")
                          .option("mergeSchema", "true")
                          .outputMode("append")                          
                          .trigger(processingTime = "10 seconds")
                          .toTable("quickstart_schema.users")
  )

## Scenario 01 - Ingesting Data

In [0]:
ingest()

## Read Table

In [0]:
from pyspark.sql.functions import col
spark.read.table("quickstart_schema.users").filter(col("id")>=5510).limit(4).display()

id,name,dob,email,gender,country,region,city,asset,marital_status,_rescued_data,education
5510,Chelsea Mcdowell,null,chelseamcdowell@ymail.com,Transgender,United States,Arkansas,Fort Smith,140903,Single,"{""dob"":""###"",""_file_path"":""abfss://users@azurestgacctraining.dfs.core.windows.net/raw/users_013_datatype_mismatch_dob.csv""}",Master's Degree
5511,Macy Holcomb,2024-10-20,macyholcomb@yahoomail.com,Transgender,India,Karnataka,Bidar,455582,Common Law,null,Bachelor's Degree


In [0]:
%sql

SELECT * from quickstart_schema.users where _rescued_data is NOT NULL;

id,name,dob,email,gender,country,region,city,asset,marital_status,_rescued_data,education
5501,Chelsea Mcdowell,null,chelseamcdowell@ymail.com,Transgender,United States,Arkansas,Fort Smith,140903,Single,"{""dob"":""###"",""_file_path"":""abfss://users@azurestgacctraining.dfs.core.windows.net/raw/users_013_datatype_mismatch_dob%20(2).csv""}",Master's Degree
5501,Chelsea Mcdowell,null,chelseamcdowell@ymail.com,Transgender,United States,Arkansas,Fort Smith,140903,Single,"{""dob"":""###"",""_file_path"":""abfss://users@azurestgacctraining.dfs.core.windows.net/raw/users_012_datatype_mismatch_dob.csv""}",Master's Degree
5501,Chelsea Mcdowell,null,chelseamcdowell@ymail.com,Transgender,United States,Arkansas,Fort Smith,140903,Single,"{""dob"":""###"",""_file_path"":""abfss://users@azurestgacctraining.dfs.core.windows.net/raw/users_012_datatype_mismatch_dob.csv""}",Master's Degree
5510,Chelsea Mcdowell,null,chelseamcdowell@ymail.com,Transgender,United States,Arkansas,Fort Smith,140903,Single,"{""dob"":""###"",""_file_path"":""abfss://users@azurestgacctraining.dfs.core.windows.net/raw/users_013_datatype_mismatch_dob.csv""}",Master's Degree


## Scenario 02 - Ingesting data with additional columns

## Scenario 03 - Ingseting data with bad records